<a href="https://colab.research.google.com/github/Aditi-24-05/mohs_hardnesss/blob/main/Mohs_hardness_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s3e25:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F60892%2F6989718%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240222%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240222T193346Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9cf0e3430a0be8f765fc7e8f89d2727083ebd6a4c2ae6315fff5f37e4b36010ed5a2d7d38ff58e9dedc971aebf21400d1c6dd945300ec9fe5a59037c281779e8160747471b28c69de1b2bc33789abb8498817a005b9ed1a8c82d43a5e456c1e20d10b1fc1a4f207c643b8ebdbb5c5e7f57b9cb53ac8530733b22547036baf35b7af3a09c5f0e4a876c3158655212038c189b6dbce0104d16ce8b18fa563965c74c42ad4aadd345a03502b57622bb77aedaeed48af1bc5d752b768b403e54a3b8adbcaeb9181877477bbc62c3fb304b872a90a61049e9fa5f66a22fa959eb1e132d75f17d506e6a62269164aa8bc8296decbeda8917d5fe3196c1dc603773090d,prediction-of-mohs-hardness-with-machine-learning:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3815527%2F6612067%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240222%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240222T193346Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0795df1aed09009573075ae0f1f44e1e2189f194189585e7ee6c849aca2ec5e179dfe578825f8931da8fdfa085befcf2788b68b2eebfd7a3a7730e13a52e0222c0592fac9d99e26f3a56bcc1763c61fc03c59529c2fc83bb94ed22ac87cf72c2979976aaecc36cb368309f9672d9c41e3aeca6263a6b7040e3c01e4155b7aec20708e5c5dc9890c6ce58a28edf8704666e67dd2d8673a80028eda5c1eb76f846def029fd69fbef6933df0287169713814c4e669a3c68abe40c424bccab444cb89ad9d01dc2716ddfed135df55c144980ab7a78cd3db95a07c85b73a5f691eb82540ec9193084d9dfb2f105f565530a175410e462a0a4b3dc98d1800288c2688e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install -q scikit-lego

# **Import Packages**


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, VotingRegressor
from sklearn.metrics import median_absolute_error, roc_auc_score, roc_curve
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import FunctionTransformer, StandardScaler, PowerTransformer
from sklearn.linear_model import QuantileRegressor
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklego.linear_model import LADRegression

# **Import data**

In [ ]:
train=pd.read_csv("/kaggle/input/playground-series-s3e25/train.csv",index_col='id')
data=pd.read_csv("/kaggle/input/prediction-of-mohs-hardness-with-machine-learning/jm79zfps6b-1/Mineral_Dataset_Supplementary_Info.csv", index_col=0)
test=pd.read_csv("/kaggle/input/playground-series-s3e25/test.csv", index_col='id')

# **Dataset Information**

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
data.head()

# **Adversarial Validation**

In [ ]:
def adversarial_validation(dataset_1=train, dataset_2=test, label='Train-Test'):

    adv_train = dataset_1.drop('Hardness', axis=1)
    adv_test = dataset_2.copy()

    adv_train['is_test'] = 0
    adv_test['is_test'] = 1

    adv = pd.concat([adv_train, adv_test], ignore_index=True)

    adv_shuffled = adv.sample(frac=1)

    adv_X = adv_shuffled.drop('is_test', axis=1)
    adv_y = adv_shuffled.is_test

    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

    val_scores = []
    predictions = np.zeros(len(adv))

    for fold, (train_idx, val_idx) in enumerate(skf.split(adv_X, adv_y)):

        adv_lr = XGBClassifier(random_state=42)
        adv_lr.fit(adv_X.iloc[train_idx], adv_y.iloc[train_idx])

        val_preds = adv_lr.predict_proba(adv_X.iloc[val_idx])[:, 1]
        predictions[val_idx] = val_preds
        val_score = roc_auc_score(adv_y.iloc[val_idx], val_preds)
        val_scores.append(val_score)

    fpr, tpr, _ = roc_curve(adv['is_test'], predictions)

    plt.figure(figsize=(15, 5))

    # ROC Curve Plot
    plt.subplot(1, 2, 1)
    sns.lineplot(x=[0, 1], y=[0, 1], linestyle="--", label="Indistinguishable Datasets")
    sns.lineplot(x=fpr, y=tpr, label="Adversarial Validation Classifier")
    plt.title(f'{label} ROC Curve\nValidation AUC = {np.mean(val_scores):.5f}', weight='bold', size=12)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

    # Distribution Plot
    plt.subplot(1, 2, 2)
    sns.kdeplot(predictions[adv['is_test'] == 0], label='Train Set')
    sns.kdeplot(predictions[adv['is_test'] == 1], label='Test Set')
    plt.title(f'{label} Predicted Probabilities Distribution', weight='bold', size=12)
    plt.xlabel('Predicted Probabilities')
    plt.ylabel('Density')

    plt.tight_layout()
    plt.show()

In [ ]:
adversarial_validation()

# **Target Distribution**

In [ ]:
plt.figure(figsize=(20, 10), dpi=300)

# Plot KDE for 'Hardness' in the 'train' dataset
sns.kdeplot(data=train, x='Hardness')

# Plot KDE for 'Hardness' in the 'orig_train' dataset
sns.kdeplot(data=data, x='Hardness')

plt.title('Hardness Distribution', fontsize=24, fontweight='bold')
plt.show()


# **Concatenation of training data and orignal data**

In [ ]:
X = pd.concat([train, data]).reset_index(drop = True).drop_duplicates()
y = X.pop('Hardness').reset_index(drop = True)

seed = 42
k = KFold(shuffle = True, random_state = seed)

np.random.seed(seed)

# **Cross validation**

In [ ]:
def cross_val_score(estimator, cv = k, label = '', include_original = True, no_duplicate = True):

    X = train.copy()
    y = X.pop('Hardness')

    #initiate prediction arrays and score lists
    val_predictions = np.zeros((len(train)))
    #train_predictions = np.zeros((len(train)))
    train_scores, val_scores = [], []

    #training model, predicting prognosis probability, and evaluating log loss
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):

        model = clone(estimator)

        #define train set
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]

        #define validation set
        X_val = X.iloc[val_idx]
        y_val = y.iloc[val_idx]

        if include_original:
            X_train = pd.concat([X_train, data.drop('Hardness', axis = 1)]).reset_index(drop = True)
            y_train = pd.concat([y_train, data.Hardness]).reset_index(drop = True)

        #remove duplicates
        if no_duplicate:
            nonduplicated_index = ~pd.concat([X_train, y_train], axis = 1).duplicated()

            y_train = y_train[nonduplicated_index]
            X_train = X_train[nonduplicated_index]

        #train model
        model.fit(X_train, y_train)

        #make predictions
        train_preds = model.predict(X_train)
        val_preds = model.predict(X_val)

        val_predictions[val_idx] += val_preds

        #evaluate model for a fold
        train_score = median_absolute_error(y_train, train_preds)
        val_score = median_absolute_error(y_val, val_preds)

        #append model score for a fold to list
        train_scores.append(train_score)
        val_scores.append(val_score)

    print(f'Val Score: {np.mean(val_scores):.5f} ± {np.std(val_scores):.5f} | Train Score: {np.mean(train_scores):.5f} ± {np.std(train_scores):.5f} | {label}')

    return val_scores, val_predictions

# **Model Evaluation**

In [ ]:
score_list, oof_list = pd.DataFrame(), pd.DataFrame()

models = [
    ('xgb_mae', XGBRegressor(random_state = seed, objective = 'reg:absoluteerror')),
    ('xgb_quant', XGBRegressor(random_state = seed, objective = 'reg:quantileerror', quantile_alpha = .5)),
    ('lgb_mae', LGBMRegressor(random_state = seed, objective = 'mae')),
    ('lgb_quant', LGBMRegressor(random_state = seed, objective = 'quantile', alpha = .5)),
    ('cb_mae', CatBoostRegressor(random_state = seed, verbose = 0, objective = 'MAE')),
    ('cb_quant', CatBoostRegressor(random_state = seed, verbose = 0, objective = 'Quantile:alpha=0.5')),
    ('gb', GradientBoostingRegressor(random_state = seed, loss = 'absolute_error')),
    ('hgb', HistGradientBoostingRegressor(random_state = seed, loss = 'absolute_error'))
]

In [ ]:
for (label, model) in models:
    score_list[label], oof_list[label] = cross_val_score(
        model,
        label=label,
        include_original=False,  # Do not include original features from 'data' in X_train
        no_duplicate=False  # You can set this to True or False based on your needs
    )

In [ ]:
plt.figure(figsize=(8, 8), dpi=300)
average_scores = score_list.mean()

# Use a gradient of colors from the Viridis colormap
colors = plt.cm.viridis(np.linspace(0, 1, len(average_scores)))

plt.pie(average_scores, labels=average_scores.index, autopct='%1.1f%%', startangle=140, colors=colors)
plt.title('Score Comparison', weight='bold', size=20)
plt.show()

# **Ensemble**

In [ ]:
weights = LADRegression().fit(oof_list, train.Hardness).coef_
pd.DataFrame(weights, index = oof_list.columns, columns = ['weight per model'])

In [ ]:
voter = VotingRegressor(models, weights = weights)
_ = cross_val_score(voter)

# **Prediction**

In [ ]:
model = clone(voter)

model.fit(X, y)
prediction = model.predict(test)

# **Submission**

In [ ]:
submission = test.copy()
submission['Hardness'] = model.predict(submission)

submission.Hardness.to_csv('submission.csv')

In [ ]:
submission.head()